In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

pd.options.mode.chained_assignment = None
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

In [34]:
d = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

d.get("https://www.babsonathletics.com/sports/w-baskbl/2022-23/schedule")

schedule_soup = BeautifulSoup(d.page_source, 'html.parser')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 110.0.5481
[WDM] - Get LATEST chromedriver version for 110.0.5481 google-chrome
[WDM] - Driver [/Users/jeremydumalig/.wdm/drivers/chromedriver/mac64/110.0.5481.77/chromedriver] found in cache


In [98]:
events = schedule_soup.find_all("tr", {"class" : "event-row"})

game_links = ["https://www.babsonathletics.com/" + e.find_all("a")[1]['href'] + "?view=boxscore" for e in events[:-1]]

game_soups = []
for url in game_links:
    d.get(url)
    game_soups.append( BeautifulSoup(d.page_source, 'html.parser') )

In [94]:
def opp_helper(soup, opp=True):
    if opp:
        opponent = [t for t in soup.find_all("div", {"class" : "player-stats"})[0].find_all("table") 
                    if t.find("h2").text != "Babson"][0]
    else:
        opponent = [t for t in soup.find_all("div", {"class" : "player-stats"})[0].find_all("table") 
                    if t.find("h2").text == "Babson"][0]

    columns = [[th.text for th in tr.find_all("th")] for tr in opponent.find_all("tr")][0][2:]
    entries = [i.text.strip() for i in opponent.find("tr", {"class" : "totals"}).find_all("td")][1:]
    
    return pd.DataFrame([entries], columns=columns)

In [124]:
results = [e.find("td", {"class" : "e_result"}).text.strip() for e in events]
results = [r for r in results if r != ""]

opponents = [e.find("td", {"class" : "e_opponent"}).text.strip().replace("\n", "") for e in events]

In [116]:
team_df = pd.concat([opp_helper(s, opp=False) for s in game_soups])
team_df = team_df.reset_index(drop=True)
team_df['Opponent'] = opponents[:-1]
opponent_df = pd.concat([opp_helper(s, opp=True) for s in game_soups])
opponent_df = opponent_df.reset_index(drop=True)
team_df['Opponent'] = opponents[:-1]

game_log = team_df.join(opponent_df, rsuffix=" OPP")

In [128]:
def fg_helper(splits):
    makes = sum([int(i.split("-")[0]) for i in splits])
    attempts = sum([int(i.split("-")[1]) for i in splits])
    
    return str(makes) + "-" + str(attempts)

def totals_helper(df):
    poss = (0.96*(sum(df['FGA'].astype(int)) + 
                  sum(df['TO'].astype(int)) + 
                  0.44*sum(df['FTA'].astype(int))) - 
            sum(df['OREB'].astype(int))) / len(df)
    opp_poss = (0.96*(sum(df['OPP FGA'].astype(int)) + 
                      sum(df['OPP TO'].astype(int)) + 
                      0.44*sum(df['OPP FTA'].astype(int))) - 
                sum(df['OPP OREB'].astype(int))) / len(df)
    ppp = round(sum(df['PTS'].astype(int)) / (poss * len(df)), 2)
    opp_ppp = round(sum(df['OPP PTS'].astype(int)) / (opp_poss * len(df)), 2)
    fgs = fg_helper( list(df['FGM/A']) )
    threes = fg_helper( list(df['3FG/A']) )
    fts = fg_helper( list(df['FTM/A']) )
    opp_fgs = fg_helper( list(df['OPP FGM/A']) )
    opp_threes = fg_helper( list(df['OPP 3PM/A']) )
    opp_fts = fg_helper( list(df['OPP FTM/A']) )
    orb = 100 * round(sum(df['OREB'].astype(int)) / (sum(df['OREB'].astype(int)) + sum(df['OPP DREB'].astype(int))), 3)
    drb = 100 * round(sum(df['DREB'].astype(int)) / (sum(df['DREB'].astype(int)) + sum(df['OPP OREB'].astype(int))), 3)
    reb = 100 * round(sum(df['REB'].astype(int)) / (sum(df['REB'].astype(int)) + sum(df['OPP REB'].astype(int))), 3)
    to = 100 * round(sum(df['TO'].astype(int)) / (poss * len(df)), 3)
    opp_to = 100 * round(sum(df['OPP TO'].astype(int)) / (opp_poss * len(df)), 3)

    return ["Total", "", sum(df['Margin']), 
            round(poss, 1), ppp, opp_ppp,
            fgs, threes, fts, opp_fgs, opp_threes, opp_fts,
            sum(df['REB'].astype(int)), sum(df['OPP REB'].astype(int)), 
            round(orb, 1), round(drb, 1), round(reb, 1), 
            round(to, 1), round(opp_to, 1)]

In [131]:
game_log['W/L'] = [r.split(",")[0] for r in results]
game_log['Margin'] = [eval(r.split(", ")[1]) for r in results]
game_log['FGA'] = game_log['FGM-A'].apply(lambda i : int(i.split("-")[1]))
game_log['FTM'] = game_log['FTM-A'].apply(lambda i : int(i.split("-")[0]))
game_log['FTA'] = game_log['FTM-A'].apply(lambda i : int(i.split("-")[1]))
game_log['OPP FGM'] = game_log['FGM-A OPP'].apply(lambda i : int(i.split("-")[0]))
game_log['OPP FGA'] = game_log['FGM-A OPP'].apply(lambda i : int(i.split("-")[1]))
game_log['OPP 3PM'] = game_log['3PM-A OPP'].apply(lambda i : int(i.split("-")[0]))
game_log['OPP 3PA'] = game_log['3PM-A OPP'].apply(lambda i : int(i.split("-")[1]))
game_log['OPP FGM/A'] = game_log['OPP FGM'].astype(str) + "-" + game_log['OPP FGA'].astype(str)
game_log['OPP 3PM/A'] = game_log['OPP 3PM'].astype(str) + "-" + game_log['OPP 3PA'].astype(str)
game_log['OPP FTM'] = game_log['FTM-A OPP'].apply(lambda i : int(i.split("-")[0]))
game_log['OPP FTA'] = game_log['FTM-A OPP'].apply(lambda i : int(i.split("-")[1]))
game_log['FTM/A'] = game_log['FTM'].astype(str) + "-" + game_log['FTA'].astype(str)
game_log['OPP FTM/A'] = game_log['OPP FTM'].astype(str) + "-" + game_log['OPP FTA'].astype(str)
game_log['OPP OREB'] = game_log['OREB OPP']
game_log['OPP DREB'] = game_log['DREB OPP']
game_log['OPP TO'] = game_log['TO OPP']
game_log['POSS'] = 0.96*(game_log['FGA'] + 
                        game_log['TO'].astype(int) + 
                        0.44*game_log['FTA'] - game_log['OREB'].astype(int))
game_log['OPP POSS'] = 0.96*(game_log['OPP FGA'] + 
                             game_log['TO OPP'].astype(int) + 
                             0.44*game_log['OPP FTA'] - game_log['OREB OPP'].astype(int))
game_log['OPP PTS'] = game_log['PTS OPP'].astype(int)
game_log['PPP'] = round(game_log['PTS'].astype(int) / game_log['POSS'], 2)
game_log['OPP PPP'] = round(game_log['PTS OPP'].astype(int) / game_log['OPP POSS'], 2)
game_log['FGM/A'] = game_log['FGM-A']
game_log['3FG/A'] = game_log['3PM-A']
game_log['OPP REB'] = game_log['REB OPP'].astype(int)
game_log['ORB%'] = 100 * round(game_log['OREB'].astype(int) / 
                          (game_log['OREB'].astype(int) + game_log['DREB OPP'].astype(int)), 3)
game_log['DRB%'] = 100 * round(game_log['DREB'].astype(int) / 
                          (game_log['DREB'].astype(int) + game_log['OREB OPP'].astype(int)), 3)
game_log['REB%'] = 100 * round(game_log['REB'].astype(int) / 
                          (game_log['REB'].astype(int) + game_log['OPP REB'].astype(int)), 3)
game_log['TO%'] = 100 * round(game_log['TO'].astype(int) / game_log['POSS'], 3)
game_log['OPP TO%'] = 100 * round(game_log['TO OPP'].astype(int) / game_log['OPP POSS'], 3)
game_log['POSS'] = game_log['POSS'].round(1)
game_log['OPP POSS'] = game_log['OPP POSS'].round(1)

totals_row = totals_helper(game_log)

game_log = game_log[['Opponent', 'W/L', 'Margin', 
                     'POSS', 'PPP', 'OPP PPP',
                     'FGM/A', '3FG/A', 'FTM/A', 'OPP FGM/A', 'OPP 3PM/A', 'OPP FTM/A',
                     'REB', 'OPP REB', 'ORB%', 'DRB%', 'REB%', 
                     'TO%', 'OPP TO%']]

game_log.loc[len(game_log)] = totals_row
game_log['Team'] = "Babson"

game_log

ValueError: Length of values (30) does not match length of index (31)

In [134]:
game_log['FGA'] = game_log['FGM/A'].apply(lambda i : int(i.split("-")[1]))
game_log['FTA'] = game_log['FTM/A'].apply(lambda i : int(i.split("-")[1]))
game_log['FTA%'] = 100 * round(game_log['FTA'] / game_log['FGA'], 3)
game_log['OPP FGA'] = game_log['OPP FGM/A'].apply(lambda i : int(i.split("-")[1]))
game_log['OPP FTA'] = game_log['OPP FTM/A'].apply(lambda i : int(i.split("-")[1]))
game_log['OPP FTA%'] = 100 * round(game_log['OPP FTA'] / game_log['OPP FGA'], 3)

game_log[['Opponent', 'W/L']]

,Opponent,W/L,Margin,POSS,PPP,OPP PPP,FGM/A,3FG/A,FTM/A,OPP FGM/A,...,REB%,TO%,OPP TO%,Team,FGA,FTA,FTA%,OPP FGA,OPP FTA,OPP FTA%
0,at Eastern Connnecticut State,W,8,60.1,1.28,1.11,32-56,4-5,9-15,25-59,...,50.0,16.6,19.2,Babson,56,15,26.8,59,16,27.1
1,at Rhode Island College,W,28,65.4,0.95,0.53,24-66,3-15,11-14,11-66,...,48.0,12.2,23.3,Babson,66,14,21.2,66,18,27.3
2,vs. Roger Williams,W,8,65.4,0.93,0.83,21-58,2-12,17-23,16-58,...,54.2,18.3,15.8,Babson,58,23,39.7,58,23,39.7
3,at Brandeis,W,29,75.3,1.13,0.72,29-72,10-24,17-17,15-55,...,56.3,15.9,25.8,Babson,72,17,23.6,55,31,56.4
4,"vs. Bridgewater State@ Williamstown, Mass.",W,22,64.4,1.21,0.85,32-66,8-18,6-16,22-62,...,55.6,21.8,30.3,Babson,66,16,24.2,62,11,17.7
5,at Williams,W,3,68.5,0.86,0.82,20-54,3-16,16-19,22-62,...,56.6,32.1,20.6,Babson,54,19,35.2,62,11,17.7
6,vs. Framingham State,W,8,71.0,1.00,0.89,26-76,8-24,11-18,24-57,...,52.9,9.9,23.9,Babson,76,18,23.7,57,16,28.1
7,"vs. Loras@ Miami Gardens, Fla.",L,21,63.1,0.82,1.15,23-57,4-23,2-4,25-60,...,47.9,22.2,12.6,Babson,57,4,7.0,60,16,26.7
8,"vs. Hanover@ Miami Gardens, Fla.",W,1,73.1,0.90,0.89,25-56,6-16,10-14,25-67,...,52.6,31.5,20.4,Babson,56,14,25.0,67,17,25.4
9,vs. Middlebury,W,9,65.5,0.92,0.79,23-57,2-13,12-12,20-55,...,62.5,32.0,24.7,Babson,57,12,21.1,55,10,18.2


In [136]:
game_log.to_csv("babson_scout.csv")